###** Education**


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote
from time import sleep
from random import randint

# Initialize lists to store the data
URL = []
Title = []
Category = []
Author_name = []
Author_url = []
Date = []
Content = []
Description = []

base_url = 'https://www.imagekhabar.com/'

# Function to get all the content of an article
def get_article_detail(article_url):
    response = requests.get(article_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract author detail
        author_box = soup.find('a', class_='uk-link-reset uk-margin-left uk-margin-right')
        if author_box:
            author_name = author_box.get_text(strip=True)
            author_url = author_box['href']
        else:
            author_name = None
            author_url = None

        # Extract date and time
        date_time_box = soup.find('div', class_='post-time uk-visible@s')
        if date_time_box:
            date_time = date_time_box.get_text(strip=True)
        else:
            date_time = None

        # Find and extract the content in <p> tags within the <article> section
        article_section = soup.find('article', class_='post-entry uk-text-center')
        if article_section:
            paragraphs = article_section.find_all('p')
            content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        else:
            content = None

        # Define description (if needed, adjust extraction logic here)
        description = None  # Update this based on your specific needs

        # Append data to lists
        Description.append(description)
        Author_name.append(author_name)
        Author_url.append(base_url + author_url if author_url else None)
        Date.append(date_time)
        Content.append(content)

# Function to scrape a specific section and a range of pages
def scrape_section(section, start_page=1, end_page=2):
    # Encode the section part of the URL
    encoded_section = quote(section)
    category = section

    sleep(randint(2, 10))
    for page in range(start_page, end_page + 1):
        url = f'https://www.imagekhabar.com/news/category/{encoded_section}/page/{page}/'
        print(f"Scraping URL: {url}")
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all article items
            article_items = soup.find_all('div', class_='uk-card-body')
            if not article_items:
                print("No articles found.")
            for article_item in article_items:
                # Extract the title and title link
                title_box = article_item.find('h3')
                if title_box:
                    title = title_box.get_text(strip=True)
                    title_link = title_box.find('a')['href']

                    # Append data to lists
                    URL.append(title_link)
                    Title.append(title)
                    Category.append(category)

                    # Scrape article details
                    get_article_detail(title_link)

                    # Sleep to avoid overwhelming the server
                    sleep(randint(2, 10))

# Example usage
scrape_section('शिक्षा', 1, 2)

# Ensure all lists are the same length
max_len = max(len(URL), len(Title), len(Category), len(Description), len(Content), len(Date), len(Author_url), len(Author_name))

# Fill shorter lists with None to match the length
URL.extend([None] * (max_len - len(URL)))
Title.extend([None] * (max_len - len(Title)))
Category.extend([None] * (max_len - len(Category)))
Description.extend([None] * (max_len - len(Description)))
Content.extend([None] * (max_len - len(Content)))
Date.extend([None] * (max_len - len(Date)))
Author_url.extend([None] * (max_len - len(Author_url)))
Author_name.extend([None] * (max_len - len(Author_name)))

# Create a DataFrame
data = {
    'URL': URL,
    'Title': Title,
    'Description': Description,
    'Content': Content,
    'Category': Category,
    'Date': Date,
    'Author_url': Author_url,
    'Author_name': Author_name
}

df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('imagekhaber_data_education.csv', index=False)
print("Data saved to scraped_data.csv")
